### Pre-installs

In [1]:
# %pip install pandas
# %pip install numpy
# %pip install matplotlib
# %pip install seaborn
# %pip install sklearn
# %pip install -U scikit-learn
# %pip install spotipy

### Inputs

In [2]:
# input_song = input("Enter your desired song name here!")
# input_playlist = input("Enter your desired playlist name here!s")
# recc_num = int(input("Enter how many songs you want to be recommended!"))
# client_id = input("Enter your Spotify API Client Key here!")
# client_secret = input("Enter your Spotify API Secret Key here!")
# sp_dc = input("Enter your Spotify sp_dc here!")
# sp_key = input("Enter your Spotify sp_key here!")

input_song = "当你老了"
input_playlist = "520"
recc_num = 10
client_id = "80e8701040fd4ee99393dbe0038161fb"
client_secret = "e5b3e975af1a4ed8adac362bddbe98f3"
sp_dc = "AQAmDvfdiz6adzBX6hjCq-bveRE5_IXVUa0SQ3CajcY5rkTUiHwmCeUU9Y0bpm6tu96J96yAvyxaLAvKPBErGwKoIN-hLra-dpONrrNYf3qxLnGT2hekc1oIn51rZPmfs1Z5qyhgWlw4efKwYH4UGnjcNtfOkU-C"
sp_key = "7e173a03-55b5-4055-9ee0-050e5a1a08e1"

### CSV & Spotify Information

In [3]:
import pandas as pd
import ast

# Importing the datasets
dataset = pd.read_csv("data/tracks_with_genres_&_language.csv")
df = pd.read_csv("data/tracks.csv")

# Creating the dataframe
df_generated = pd.DataFrame(dataset)
df_generated['genres'] = df_generated['genres'].apply(ast.literal_eval)

In [4]:
df_generated.iloc[:,3:19].head()

,duration_ms,explicit,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,160187,0,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.484,0.265,0,-11.101,1,0.0322,0.394,0.000000,0.149,0.285,113.564,3
1,159560,0,['6lH5PpuiMa5SpfjoIOlwCS'],2020-03-13,0.671,0.867,2,-2.706,1,0.0571,0.436,0.000000,0.139,0.839,120.689,4
2,141987,0,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.405,0.365,6,-10.226,0,0.0289,0.255,0.000005,0.163,0.588,104.536,4
3,157093,0,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.477,0.352,1,-14.165,1,0.0300,0.406,0.000000,0.122,0.478,106.773,4
4,187333,0,['1Mxqyy3pSjf8kZZL4QVxS0'],2018-05-04,0.319,0.201,7,-17.796,1,0.0623,0.887,0.000000,0.904,0.239,117.153,3


In [5]:
# Spotify API Authentication Information
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
# api key
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id,
                                               client_secret=client_secret,
                                               redirect_uri="http://localhost/",
                                               scope="playlist-modify-private",
                                               show_dialog=True,
                                               cache_path="token.txt"
                                               ))

In [6]:
import seaborn as sns
import matplotlib.pyplot as plt

# # Assuming df_generated is your DataFrame
# # Adjust this part based on your actual DataFrame structure
# # For example, you might need to select specific columns or rows
# # or convert categorical variables to numerical values for visualization

# df_generated.columns
# df_graph = df_generated[['popularity', 'duration_ms', 'explicit',
#        'danceability', 'energy', 'key', 'loudness',
#        'speechiness', 'acousticness', 'instrumentalness', 'liveness',
#        'valence', 'tempo', 'time_signature']]

# sns.set(style="white")  # Set the style of the plot

# # Create a heatmap using seaborn with annotated values for the entire correlation matrix
# plt.figure(figsize=(12, 10))  # Adjust the figure size as needed
# heatmap = sns.heatmap(df_graph.corr(numeric_only=True), annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)

# # Customize the appearance of the heatmap
# heatmap.set_title('Heatmap of df_generated', fontsize=16)

# plt.show()

### Processing the Input Song

In [7]:
# Using Spotify API to search for a song's information based on input and adding the necessary information in a DataFrame.
def search_track(track_name):
    # Search for the track
    results = sp.search(q=track_name, type='track')

    # Check if the track exists
    if results['tracks']['total'] > 0:
        # Get the first track from the results
        track = results['tracks']['items'][0]
        explicit = int(track["explicit"] == True)
        artists = []
        id_artists = []
        for i in range (len(track["artists"])):
            artists.append(track["artists"][i]["name"])
            id_artists.append(track["artists"][i]["id"])

        print(f'Found track: {track["name"]} by {track["artists"][0]["name"]} from the album {track["album"]["name"]}.')
        track_dict = {"id": track["id"], "name": track["name"], "popularity": track["popularity"], 
                      "duration_ms": track["duration_ms"], "explicit": explicit, "artists": str(artists),
                      "id_artists": str(id_artists), "release_date": track["album"]["release_date"]}
        return track_dict
    else:
        print('Track not found')
        return None

# Creating a Single Row DataFrame for the input song.
track_result = search_track(input_song)
td = pd.DataFrame(track_result, index=[0])
td

Found track: 当你老了 by Karen Mok from the album 当你老了.


,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date
0,5KsMzI24fitxacmN4NABMC,当你老了,40,198263,0,['Karen Mok'],['6jlz5QSUqbKE4vnzo2qfP1'],2015-02-18


In [8]:
# Obtaining Feature Data from song based on its song_id from previous function and adding them to a DataFrame.
def get_audio_features(track_result):
    song_id = track_result["id"]
    results = sp.audio_features(song_id)

    if results:
        return results[0]
    else:
        print(f'No audio features found for song ID: {song_id}')
        return None

audio_features = get_audio_features(track_result)
af = pd.DataFrame(audio_features, index=[0])
# Taking out the irrevelant features.
af_formatted = af.drop(["type", "id", "uri", "track_href", "analysis_url", "duration_ms"], axis=1)
# Merging both DataFrames to sync up with the dataset's layout.
td = pd.concat([td, af_formatted], axis=1)
td['year'] = td['release_date'].str.extract(r'(\d{4})').astype(int)

In [9]:
year_priority = td['year'][0]
year_priority

2015

In [10]:
import ast

# Converting string array into regular array.
td['artists'] = td['artists'].apply(ast.literal_eval)

td

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year
0,5KsMzI24fitxacmN4NABMC,当你老了,40,198263,0,[Karen Mok],['6jlz5QSUqbKE4vnzo2qfP1'],2015-02-18,0.233,0.229,...,-9.346,1,0.0363,0.93,0.000048,0.118,0.172,176.924,5,2015


In [11]:
# Viewing the input song features.
td['artists_count'] = len(td['artists'][0])
td

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year,artists_count
0,5KsMzI24fitxacmN4NABMC,当你老了,40,198263,0,[Karen Mok],['6jlz5QSUqbKE4vnzo2qfP1'],2015-02-18,0.233,0.229,...,1,0.0363,0.93,0.000048,0.118,0.172,176.924,5,2015,1


In [12]:
# Extracting the artists from the input artists array.
expanded_artists = td['artists'].apply(lambda x: pd.Series(x))

# Rename the columns
expanded_artists.columns = [f"artist_{i+1}" for i in range(len(expanded_artists.columns))]

# Concatenate the expanded columns with the original DataFrame
td = pd.concat([td, expanded_artists], axis=1)
td.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year,artists_count,artist_1
0,5KsMzI24fitxacmN4NABMC,当你老了,40,198263,0,[Karen Mok],['6jlz5QSUqbKE4vnzo2qfP1'],2015-02-18,0.233,0.229,...,0.0363,0.93,0.000048,0.118,0.172,176.924,5,2015,1,Karen Mok


In [13]:
# Obtaining input artist(s) genre(s) through the Spotify API.
def get_artist_genres(artist_name):
    try:
        results = sp.search(q='artist:' + artist_name, type='artist')
        genres = results['artists']['items'][0]['genres']
        return genres if genres else []
    except IndexError:
        return []
    
for i in range(td['artists_count'][0]):
    td['genres' + str(i+1)] = td['artist_' + str(i+1)].apply(get_artist_genres)
    td['genres' + str(i+1)] = td['genres' + str(i+1)].fillna('[]')
td

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year,artists_count,artist_1,genres1
0,5KsMzI24fitxacmN4NABMC,当你老了,40,198263,0,[Karen Mok],['6jlz5QSUqbKE4vnzo2qfP1'],2015-02-18,0.233,0.229,...,0.93,0.000048,0.118,0.172,176.924,5,2015,1,Karen Mok,[mandopop]


In [14]:
# Combining all genres into one list.
for i in range(td['artists_count'][0]):
    td['genres'] = td['genres' + str(i+1)] + td['genres' + str(i+1)]
    td.drop(['genres' + str(i+1)], axis=1, inplace=True)
td

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year,artists_count,artist_1,genres
0,5KsMzI24fitxacmN4NABMC,当你老了,40,198263,0,[Karen Mok],['6jlz5QSUqbKE4vnzo2qfP1'],2015-02-18,0.233,0.229,...,0.93,0.000048,0.118,0.172,176.924,5,2015,1,Karen Mok,"[mandopop, mandopop]"


In [15]:
# Returning only the unique genres for the input.
def get_unique(row):
    return list(set(row))

td['genres'] = td['genres'].apply(get_unique)

In [16]:
# Determining the language from a Lyrics Scraping Library.
import spotify_lyrics_scraper as spotify

token = spotify.getToken(sp_dc, sp_key)
lyrics_data = spotify.getLyrics(token, trackId=td['id'][0])
if lyrics_data['status'] == False:
        language = "null"
else:
    # Extract the language from the lyrics data
    language = lyrics_data['message']['lyrics']['language']

td['language'] = language
td

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,instrumentalness,liveness,valence,tempo,time_signature,year,artists_count,artist_1,genres,language
0,5KsMzI24fitxacmN4NABMC,当你老了,40,198263,0,[Karen Mok],['6jlz5QSUqbKE4vnzo2qfP1'],2015-02-18,0.233,0.229,...,0.000048,0.118,0.172,176.924,5,2015,1,Karen Mok,[mandopop],z1


In [17]:
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException

# Many songs do not have lyrics enabled by spotify. So the alternative to that is to use their name as a benchmark for the language.
# Uses language detect library to identify the song name's expected language. Not exactly accurate but it is an acceptable alternative for now until NLP.
def replace_nan_with_language(row):
    if pd.isna(row['language']):  # Check if the value is NaN
        name = row['name']
        if isinstance(name, str) and len(name) > 0:  # Check if name is a non-empty string
            try:
                return detect(name)  # Use detect() output as the value if it's NaN
            except LangDetectException:
                pass  # Handle LangDetectException, e.g., return 'unknown' or None
        # Return None or any other default value if language detection fails or input text is empty
        return None  
    else:
        return row['language']  # Otherwise, keep the original value

td['language'] = td.apply(replace_nan_with_language, axis=1)

td.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,instrumentalness,liveness,valence,tempo,time_signature,year,artists_count,artist_1,genres,language
0,5KsMzI24fitxacmN4NABMC,当你老了,40,198263,0,[Karen Mok],['6jlz5QSUqbKE4vnzo2qfP1'],2015-02-18,0.233,0.229,...,0.000048,0.118,0.172,176.924,5,2015,1,Karen Mok,[mandopop],z1


In [18]:
# Creating a unique identifier.
td['song_name_artist'] = td['name'] + str(td['artists'])

In [19]:
# Defining the values to give priority to later.
language_prority = td['language'][0]
if (language_prority == 'z1') or (language_prority == 'zh-cn') or (language_prority == 'zh-tw'):
    language_prority = 'zh'
    
key_priority = td['key'][0]
genre_priority = td['genres'][0]

print(language_prority, key_priority, genre_priority)

zh 0 ['mandopop']


### Adding Input to Dataset and further processing

In [20]:
# Adding the Input Song to the Dataset DataFrame. Added to the very front.
new_df = pd.concat([td, df_generated], ignore_index=True)
new_df.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,time_signature,year,artists_count,artist_1,genres,language,song_name_artist,artist_2,artist_3,artist_4
0,5KsMzI24fitxacmN4NABMC,当你老了,40,198263,0,[Karen Mok],['6jlz5QSUqbKE4vnzo2qfP1'],2015-02-18,0.233,0.229,...,5,2015,1,Karen Mok,[mandopop],z1,"当你老了0 [Karen Mok]\nName: artists, dtype: ob...",NaN,NaN,NaN
1,6catF1lDhNTjjGa2GxRQNN,You'll Never Walk Alone - Mono; 2002 Remaster,56,160187,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.484,0.265,...,3,2008,1,Gerry & The Pacemakers,"[adult standards, bubblegum pop, merseybeat, r...",en,You'll Never Walk Alone - Mono; 2002 Remaster[...,NaN,NaN,NaN
2,6Pkt6qVikqPBt9bEQy8iTz,A Lover's Concerto,41,159560,0,NaN,['6lH5PpuiMa5SpfjoIOlwCS'],2020-03-13,0.671,0.867,...,4,2020,1,The Toys,[],en,A Lover's Concerto['The Toys'],NaN,NaN,NaN
3,4aSw1QJIMwYSoDEgzgdCJL,Ferry Cross the Mersey - Mono; 2002 Remaster,40,141987,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.405,0.365,...,4,2008,1,Gerry & The Pacemakers,"[adult standards, bubblegum pop, merseybeat, r...",en,Ferry Cross the Mersey - Mono; 2002 Remaster['...,NaN,NaN,NaN
4,0ZMMtH875IR2TfkyC4PolD,Don't Let the Sun Catch You Crying (Main) - Mono,34,157093,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.477,0.352,...,4,2008,1,Gerry & The Pacemakers,"[adult standards, bubblegum pop, merseybeat, r...",en,Don't Let the Sun Catch You Crying (Main) - Mo...,NaN,NaN,NaN


In [21]:
new_df['langauge'] = new_df['language'].fillna("null")
new_df.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,year,artists_count,artist_1,genres,language,song_name_artist,artist_2,artist_3,artist_4,langauge
0,5KsMzI24fitxacmN4NABMC,当你老了,40,198263,0,[Karen Mok],['6jlz5QSUqbKE4vnzo2qfP1'],2015-02-18,0.233,0.229,...,2015,1,Karen Mok,[mandopop],z1,"当你老了0 [Karen Mok]\nName: artists, dtype: ob...",NaN,NaN,NaN,z1
1,6catF1lDhNTjjGa2GxRQNN,You'll Never Walk Alone - Mono; 2002 Remaster,56,160187,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.484,0.265,...,2008,1,Gerry & The Pacemakers,"[adult standards, bubblegum pop, merseybeat, r...",en,You'll Never Walk Alone - Mono; 2002 Remaster[...,NaN,NaN,NaN,en
2,6Pkt6qVikqPBt9bEQy8iTz,A Lover's Concerto,41,159560,0,NaN,['6lH5PpuiMa5SpfjoIOlwCS'],2020-03-13,0.671,0.867,...,2020,1,The Toys,[],en,A Lover's Concerto['The Toys'],NaN,NaN,NaN,en
3,4aSw1QJIMwYSoDEgzgdCJL,Ferry Cross the Mersey - Mono; 2002 Remaster,40,141987,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.405,0.365,...,2008,1,Gerry & The Pacemakers,"[adult standards, bubblegum pop, merseybeat, r...",en,Ferry Cross the Mersey - Mono; 2002 Remaster['...,NaN,NaN,NaN,en
4,0ZMMtH875IR2TfkyC4PolD,Don't Let the Sun Catch You Crying (Main) - Mono,34,157093,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.477,0.352,...,2008,1,Gerry & The Pacemakers,"[adult standards, bubblegum pop, merseybeat, r...",en,Don't Let the Sun Catch You Crying (Main) - Mo...,NaN,NaN,NaN,en


In [22]:
new_df['song_id_name'] = new_df['id'] + new_df['name']
print(new_df.shape)
new_df.drop_duplicates(subset=["song_id_name"], keep='first', inplace=True)
print(new_df.shape)

(186012, 31)


(186012, 31)


In [23]:
# Counting the number of genres in for each song.
def count_items_in_list(lst):
    return len(lst)

# Apply the function to the DataFrame column
new_df['genres_count'] = new_df['genres'].apply(count_items_in_list)
new_df.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,artist_1,genres,language,song_name_artist,artist_2,artist_3,artist_4,langauge,song_id_name,genres_count
0,5KsMzI24fitxacmN4NABMC,当你老了,40,198263,0,[Karen Mok],['6jlz5QSUqbKE4vnzo2qfP1'],2015-02-18,0.233,0.229,...,Karen Mok,[mandopop],z1,"当你老了0 [Karen Mok]\nName: artists, dtype: ob...",NaN,NaN,NaN,z1,5KsMzI24fitxacmN4NABMC当你老了,1
1,6catF1lDhNTjjGa2GxRQNN,You'll Never Walk Alone - Mono; 2002 Remaster,56,160187,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.484,0.265,...,Gerry & The Pacemakers,"[adult standards, bubblegum pop, merseybeat, r...",en,You'll Never Walk Alone - Mono; 2002 Remaster[...,NaN,NaN,NaN,en,6catF1lDhNTjjGa2GxRQNNYou'll Never Walk Alone ...,9
2,6Pkt6qVikqPBt9bEQy8iTz,A Lover's Concerto,41,159560,0,NaN,['6lH5PpuiMa5SpfjoIOlwCS'],2020-03-13,0.671,0.867,...,The Toys,[],en,A Lover's Concerto['The Toys'],NaN,NaN,NaN,en,6Pkt6qVikqPBt9bEQy8iTzA Lover's Concerto,0
3,4aSw1QJIMwYSoDEgzgdCJL,Ferry Cross the Mersey - Mono; 2002 Remaster,40,141987,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.405,0.365,...,Gerry & The Pacemakers,"[adult standards, bubblegum pop, merseybeat, r...",en,Ferry Cross the Mersey - Mono; 2002 Remaster['...,NaN,NaN,NaN,en,4aSw1QJIMwYSoDEgzgdCJLFerry Cross the Mersey -...,9
4,0ZMMtH875IR2TfkyC4PolD,Don't Let the Sun Catch You Crying (Main) - Mono,34,157093,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.477,0.352,...,Gerry & The Pacemakers,"[adult standards, bubblegum pop, merseybeat, r...",en,Don't Let the Sun Catch You Crying (Main) - Mo...,NaN,NaN,NaN,en,0ZMMtH875IR2TfkyC4PolDDon't Let the Sun Catch ...,9


In [24]:
print(new_df.shape)
new_df = new_df[(new_df['genres_count'] != 0)]
print(new_df.shape)

(186012, 32)
(166252, 32)


In [25]:
new_df['language'] = new_df['language'].apply(lambda row: 'zh' if (row == 'z1') or (row == 'zh-cn') or (row == 'zh-tw') else row)

In [26]:
new_df['language'].unique()

array(['zh', 'en', 'de', 'ja', 'da', 'sl', 'cy', 'pt', 'id', 'af', 'nl',
       'ko', 'sv', 'tl', 'hu', 'no', 'fr', 'so', 'sq', 'fi', 'it', 'es',
       'tr', 'ro', 'et', 'sk', 'sw', 'lv', 'cs', 'lt', 'ca', 'pl', 'mk',
       'hr', 'ar', 'ru', nan, 'hi', 'ms', 'th', 'nn', 'gl', 'vi', 'ug',
       'm7', 'ia', 'co', 'eu', 'tt', 'el', 'ta', 'bg', 'zu', 'xh', 'st',
       'sn', 'tn', 'gn', 'sm', 'rn', 'fa', 'br', 'he', 'is', 'ne', 'sr',
       'ln', 'la', 'mi', 'ht', 'az', 'qu', 'iw', 'uk', 'am', 'ur'],
      dtype=object)

### Supersetting genres

In [27]:
# Determining the frequency of the genre. Giving the input song's genre priority to go first.
genre_extract = new_df['genres'].reset_index()
genre_wo_index = genre_extract['genres']
tally = genre_wo_index.count()
genre_dict = {}

for i in range (tally):
    input = genre_wo_index.loc[i]
    if len(input) > 0:
        for n in input:
            if n in genre_dict.keys():
                genre_dict[n] += 1
            else:
                genre_dict[n] = 1

print(genre_dict)

{'mandopop': 3642, 'adult standards': 1216, 'bubblegum pop': 416, 'merseybeat': 529, 'rock-and-roll': 650, 'british invasion': 495, 'classic uk pop': 704, 'brill building pop': 763, 'rockabilly': 550, 'folk rock': 933, 'easy listening': 126, 'lounge': 283, 'british blues': 265, 'canadian blues': 54, 'singer-songwriter': 312, 'lilith': 307, 'canadian singer-songwriter': 107, 'folk': 393, 'rock': 3193, 'mellow gold': 1141, 'jazz': 304, 'swing': 56, 'big band': 94, 'jazz piano': 73, 'stride': 75, 'vocal jazz': 355, 'cool jazz': 274, 'harlem renaissance': 23, 'jazz saxophone': 71, 'contemporary jazz': 73, 'jazz quartet': 20, 'jazz fusion': 297, 'swedish jazz': 134, 'free jazz': 58, 'avant-garde jazz': 49, 'contemporary post-bop': 57, 'italian jazz': 15, 'soundtrack': 456, 'italian soundtrack': 28, 'vintage italian soundtrack': 34, 'classic soundtrack': 89, 'jazz trumpet': 176, 'hard bop': 204, 'roots rock': 250, 'psychedelic rock': 592, 'classic rock': 1358, 'acid rock': 67, 'electric blue

In [28]:
genres_df = pd.Series(genre_dict)
genres_df.head()

mandopop           3642
adult standards    1216
bubblegum pop       416
merseybeat          529
rock-and-roll       650
dtype: int64

In [29]:
# Summary Statistics of genres.
genres_df.T.describe()

count    4229.000000
mean      130.132419
std       393.312314
min         1.000000
25%         4.000000
50%        19.000000
75%        85.000000
max      6880.000000
dtype: float64

In [30]:
# genres_df.to_csv("data/genre_frequency.csv")

In [31]:
# Creating the genre supersets, with priority to the input song's genre.
# Since the input song is the first one to be run, its genre will be able to start the classification of other sub-genres under it.

# This can be further improved. Allowing for later genres to take over earlier genres as the superset if it more general.
# E.g.: Detroit Hip Hop is found before general hip hop. Currently they will be seperated into 2 seperate genres because the first word 'detroit' is
# not 'hip hop' and is thus overlooked.

def generate_superset_mapping(genre_dict):
    superset_mapping = {}

    for genre, frequency in genre_dict.items():
        subset_added = False
        for superset in list(superset_mapping.keys()):
            if genre in superset:
                superset_mapping[genre] = [genre]
                superset_mapping[genre].extend(superset_mapping[superset])
                del superset_mapping[superset]
                subset_added = True
                break
            elif superset in genre:
                if superset in superset_mapping:
                    superset_mapping[superset].append(genre)
                    subset_added = True
                    break
        if not subset_added:
            superset_mapping[genre] = [genre]

    return superset_mapping

# Generate superset mapping
superset_mapping = generate_superset_mapping(genre_dict)

# Output the superset mapping
for superset, subsets in superset_mapping.items():
    print(f"{superset}: {subsets}")


adult standards: ['adult standards', 'deep adult standards']
bubblegum pop: ['bubblegum pop']
merseybeat: ['merseybeat']
british invasion: ['british invasion']
brill building pop: ['brill building pop']
rockabilly: ['rockabilly', 'uk rockabilly', 'finnish rockabilly', 'german rockabilly', 'neo-rockabilly', 'swedish rockabilly', 'spanish rockabilly', 'rockabilly en espanol', 'japanese rockabilly', 'canadian rockabilly']
easy listening: ['easy listening']
lounge: ['lounge', 'lounge house', 'chill lounge', 'italian lounge', 'sunset lounge']
canadian blues: ['canadian blues']
singer-songwriter: ['singer-songwriter', 'canadian singer-songwriter', 'nashville singer-songwriter', 'taiwan singer-songwriter', 'singaporean singer-songwriter', 'indian singer-songwriter', 'scottish singer-songwriter', 'japanese singer-songwriter', 'swedish singer-songwriter', 'british singer-songwriter', 'turkish singer-songwriter', 'irish singer-songwriter', 'norwegian singer-songwriter', 'nz singer-songwriter', '

In [32]:
# Supersetting the weighted genres for later.

superset_genre_priority = []
if len(genre_priority) != 0:
    for i in genre_priority:
        for n in superset_mapping:
            if i in n:
                superset_genre_priority.append(n)
print(superset_genre_priority)

[]


In [33]:
# Now with the superset mapped out, apply it onto the all the songs to determine their supersetted genre(s).
# This can be improved. Instead of only returning the first superset genre, it should return multiple in an array.

import pandas as pd

new_df['supersetted_genres'] = new_df['genres'].apply(lambda genres: [m for n in genres for m in superset_mapping if n in m])

In [34]:
new_df['supersetted_genres'].value_counts().count()

6117

In [35]:
new_df.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,genres,language,song_name_artist,artist_2,artist_3,artist_4,langauge,song_id_name,genres_count,supersetted_genres
0,5KsMzI24fitxacmN4NABMC,当你老了,40,198263,0,[Karen Mok],['6jlz5QSUqbKE4vnzo2qfP1'],2015-02-18,0.233,0.229,...,[mandopop],zh,"当你老了0 [Karen Mok]\nName: artists, dtype: ob...",NaN,NaN,NaN,z1,5KsMzI24fitxacmN4NABMC当你老了,1,[]
1,6catF1lDhNTjjGa2GxRQNN,You'll Never Walk Alone - Mono; 2002 Remaster,56,160187,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.484,0.265,...,"[adult standards, bubblegum pop, merseybeat, r...",en,You'll Never Walk Alone - Mono; 2002 Remaster[...,NaN,NaN,NaN,en,6catF1lDhNTjjGa2GxRQNNYou'll Never Walk Alone ...,9,"[adult standards, bubblegum pop, merseybeat, b..."
3,4aSw1QJIMwYSoDEgzgdCJL,Ferry Cross the Mersey - Mono; 2002 Remaster,40,141987,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.405,0.365,...,"[adult standards, bubblegum pop, merseybeat, r...",en,Ferry Cross the Mersey - Mono; 2002 Remaster['...,NaN,NaN,NaN,en,4aSw1QJIMwYSoDEgzgdCJLFerry Cross the Mersey -...,9,"[adult standards, bubblegum pop, merseybeat, b..."
4,0ZMMtH875IR2TfkyC4PolD,Don't Let the Sun Catch You Crying (Main) - Mono,34,157093,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.477,0.352,...,"[adult standards, bubblegum pop, merseybeat, r...",en,Don't Let the Sun Catch You Crying (Main) - Mo...,NaN,NaN,NaN,en,0ZMMtH875IR2TfkyC4PolDDon't Let the Sun Catch ...,9,"[adult standards, bubblegum pop, merseybeat, b..."
5,1hx7X9cMXHWJjknb9O6Ava,The September Of My Years - Live At The Sands ...,26,187333,0,NaN,['1Mxqyy3pSjf8kZZL4QVxS0'],2018-05-04,0.319,0.201,...,"[easy listening, adult standards, lounge]",en,The September Of My Years - Live At The Sands ...,NaN,NaN,NaN,en,1hx7X9cMXHWJjknb9O6AvaThe September Of My Year...,3,"[easy listening, adult standards, lounge]"


### Encoding

In [36]:
# # One-hot Encoding of all the superset genres of each song in the dataframe.

# genres_dummies = pd.get_dummies(new_df['supersetted_genres'].apply(pd.Series).stack(), prefix="genre").groupby(level=0).sum()

# # Concatenate the binary columns with the original DataFrame
# df_2 = pd.concat([new_df, genres_dummies], axis=1)

In [37]:
# df_2.iloc[:,0:20].describe()

In [38]:
for i in new_df.loc[0]['artists']:
    for index, row in new_df.iterrows():
        # Check if the value of x is in any of the artist columns
        if i in [row['artist_1'], row['artist_2'], row['artist_3'], row['artist_4']]:
            # If x is found in any of the artist columns, set the value of matching_artist column to 1
            new_df.at[index, 'matching_artist'] = 1
        else:
            new_df.at[index, 'matching_artist'] = 0
new_df['matching_artist'].head()

0    1.0
1    0.0
3    0.0
4    0.0
5    0.0
Name: matching_artist, dtype: float64

In [39]:
# Determining how many keys there are.
new_df.groupby('key').count()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,language,song_name_artist,artist_2,artist_3,artist_4,langauge,song_id_name,genres_count,supersetted_genres,matching_artist
key,,,,,,,,,,,,,,,,,,,,,
0,19587,19587,19587,19587,19587,1,19587,19587,19587,19587,...,19536,19587,4082,1013,194,19587,19587,19587,19587,19587
1,14502,14502,14502,14502,14502,0,14502,14502,14502,14502,...,14446,14502,3955,1026,220,14502,14502,14502,14502,14502
2,17337,17337,17337,17337,17337,0,17337,17337,17337,17337,...,17297,17337,3440,921,164,17337,17337,17337,17337,17337
3,5024,5024,5024,5024,5024,0,5024,5024,5024,5024,...,5004,5024,1090,271,37,5024,5024,5024,5024,5024
4,13626,13626,13626,13626,13626,0,13626,13626,13626,13626,...,13586,13626,2714,646,126,13626,13626,13626,13626,13626
5,13199,13199,13199,13199,13199,0,13199,13199,13199,13199,...,13165,13199,2882,709,162,13199,13199,13199,13199,13199
6,11252,11252,11252,11252,11252,0,11252,11252,11252,11252,...,11208,11252,2821,708,152,11252,11252,11252,11252,11252
7,19669,19669,19669,19669,19669,0,19669,19669,19669,19669,...,19618,19669,4162,1053,215,19669,19669,19669,19669,19669
8,10079,10079,10079,10079,10079,0,10079,10079,10079,10079,...,10042,10079,2474,646,140,10079,10079,10079,10079,10079


In [40]:
new_df['key'].max()

11

In [41]:
# One-hot encoding the keys.
key_encoded = pd.get_dummies(new_df['key'], columns=[i for i in range(12)], prefix='key').astype(int)

In [42]:
key_encoded.head()

,key_0,key_1,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11
0,1,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,1,0,0,0,0


In [43]:
# Adding keys encoding into the dataset.
df_2 = pd.concat([new_df, key_encoded], axis=1)

In [44]:
print(df_2.shape)
df_2.drop_duplicates(subset=["song_id_name"], keep='first', inplace=True)
print(df_2.shape)

(166252, 46)
(166252, 46)


In [45]:
# # Checking all the languages.
# df_2['langauge'].value_counts().keys()

In [46]:
# df_2['language'].head()

In [47]:
# # Encoding all the languages.
# df_2['language'] = df_2['language'].fillna('null')
# language_encoded = pd.get_dummies(df_2['language'], prefix='lang').astype(int)
# language_encoded.head()

In [48]:
# language_encoded = language_encoded.drop(['lang_null'], axis=1)
# language_encoded.head()

In [49]:
# # Adding language encoding into the dataset.
# df_2 = pd.concat([df_2, language_encoded], axis=1)

In [50]:
# print(df_2.shape)
# df_2.drop_duplicates(subset=["song_id_name"], keep='first', inplace=True)
# print(df_2.shape)

In [51]:
# Removing unnecessary features.
df_2 = df_2.drop(['id_artists', 'release_date', 'popularity', 'key', 'artists_count', 'song_name_artist', 'artist_1', 'artist_2', 'artist_3', 'artist_4', 'genres', 'explicit', 'time_signature'], axis=1)
df_2.head()

,id,name,duration_ms,artists,danceability,energy,loudness,mode,speechiness,acousticness,...,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11
0,5KsMzI24fitxacmN4NABMC,当你老了,198263,[Karen Mok],0.233,0.229,-9.346,1,0.0363,0.930,...,0,0,0,0,0,0,0,0,0,0
1,6catF1lDhNTjjGa2GxRQNN,You'll Never Walk Alone - Mono; 2002 Remaster,160187,NaN,0.484,0.265,-11.101,1,0.0322,0.394,...,0,0,0,0,0,0,0,0,0,0
3,4aSw1QJIMwYSoDEgzgdCJL,Ferry Cross the Mersey - Mono; 2002 Remaster,141987,NaN,0.405,0.365,-10.226,0,0.0289,0.255,...,0,0,0,0,1,0,0,0,0,0
4,0ZMMtH875IR2TfkyC4PolD,Don't Let the Sun Catch You Crying (Main) - Mono,157093,NaN,0.477,0.352,-14.165,1,0.0300,0.406,...,0,0,0,0,0,0,0,0,0,0
5,1hx7X9cMXHWJjknb9O6Ava,The September Of My Years - Live At The Sands ...,187333,NaN,0.319,0.201,-17.796,1,0.0623,0.887,...,0,0,0,0,0,1,0,0,0,0


In [52]:
# Filling in any null values. (In case)

df_2 = df_2.fillna(0)
df_2.head()

,id,name,duration_ms,artists,danceability,energy,loudness,mode,speechiness,acousticness,...,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11
0,5KsMzI24fitxacmN4NABMC,当你老了,198263,[Karen Mok],0.233,0.229,-9.346,1,0.0363,0.930,...,0,0,0,0,0,0,0,0,0,0
1,6catF1lDhNTjjGa2GxRQNN,You'll Never Walk Alone - Mono; 2002 Remaster,160187,0,0.484,0.265,-11.101,1,0.0322,0.394,...,0,0,0,0,0,0,0,0,0,0
3,4aSw1QJIMwYSoDEgzgdCJL,Ferry Cross the Mersey - Mono; 2002 Remaster,141987,0,0.405,0.365,-10.226,0,0.0289,0.255,...,0,0,0,0,1,0,0,0,0,0
4,0ZMMtH875IR2TfkyC4PolD,Don't Let the Sun Catch You Crying (Main) - Mono,157093,0,0.477,0.352,-14.165,1,0.0300,0.406,...,0,0,0,0,0,0,0,0,0,0
5,1hx7X9cMXHWJjknb9O6Ava,The September Of My Years - Live At The Sands ...,187333,0,0.319,0.201,-17.796,1,0.0623,0.887,...,0,0,0,0,0,1,0,0,0,0


In [53]:
df_2.shape

(166252, 33)

### Weighting

In [54]:
numericals = {'danceability': 0, 'energy': 0.15, 'loudness': 0.1, 'mode': 0.10, 'speechiness': 0.05, 'acousticness': 0.20, 
    'instrumentalness': 0.25, 'liveness': 0.05, 'valence': 0.15, 'tempo': 0.1}
categoricals = {'key_': 0.05}
numericals_key = numericals.keys()
categoricals_key = categoricals.keys()

In [55]:
from sklearn.preprocessing import MinMaxScaler
# Normalize numerical features
numerical_features = ['loudness', 'tempo']
scaler = MinMaxScaler()
df_2[numerical_features] = scaler.fit_transform(df_2[numerical_features])

# Standardize Year
df_2['standardized_year'] = scaler.fit_transform(df_2[['year']])


In [56]:
for i in numericals_key:
    df_2[i] = df_2[i] * numericals[i]

In [57]:
for n in categoricals_key:
    for z in df_2.columns:
        if n in z:
            df_2[z] = df_2[z] * categoricals[n]

In [58]:
# # Applying weight to input language.
# df_2['lang_' + language_prority] = df_2['lang_' + language_prority] * 10
# df_2[['name','lang_' + language_prority]].head()

In [59]:
# # Reducing weight from non-input language.
# for i in df_2.columns:
#     if 'lang_' in i:
#         if i != ('lang_' + language_prority):
#             df_2[i] = df_2[i] / 10
# df_2.head()

In [60]:
# # Applying weight to input key.
# df_2['key_' + str(key_priority)] = df_2['key_' + str(key_priority)] * 10
# df_2[['name','key_' + str(key_priority)]].head()

In [61]:
# # Reducing weight from non-input key.
# for i in df_2.columns:
#     if 'key_' in i:
#         if i != ('key_' + str(key_priority)):
#             df_2[i] = df_2[i] / 10
# df_2['key_7'].head()

In [62]:
# # Applying weight to input genre(s).
# superset_genre_priority = list(set(superset_genre_priority))

# if len(superset_genre_priority) != 0:
#     for i in superset_genre_priority:
#         df_2[i] = df_2[i] * 10
# df_2[superset_genre_priority].head()

In [63]:
# # Reducing weight from non-input genre(s).
# for n in superset_genre_priority:
#     for i in df_2.columns:
#         if i in superset_mapping.keys():
#             if i != (n):
#                 df_2[i] = df_2[i] / 10
# df_2['pop'].head()

### Clustering & K Nearest Neighbours

In [64]:
df_features = df_2[['energy', 'valence', 'acousticness']]

In [65]:
from sklearn.cluster import KMeans

# Kmeans model
kmeans = KMeans(n_clusters = 3, random_state = 300, n_init = 20, tol = 0.0001)
# Fit and predict on the data
y_kmeans = kmeans.fit_predict(df_features)
#Save centroids
centroids = kmeans.cluster_centers_
# Save the predictions as a column
df_2['y_kmeans'] = y_kmeans
# Check the distribution
df_2['y_kmeans'].value_counts()

y_kmeans
0    63090
2    55743
1    47419
Name: count, dtype: int64

In [66]:
from sklearn.neighbors import KNeighborsClassifier

neigh_X = df_2[['danceability', 'energy', 'loudness',
       'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo']]
neigh_y = df_2['y_kmeans']

neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(neigh_X, neigh_y)

cluster_values = td[['danceability', 'energy', 'loudness',
       'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo']]
print(neigh.predict(cluster_values))
print(neigh.predict_proba(cluster_values))

cluster_neigh = neigh.predict(cluster_values)[0]

[2]
[[0.         0.33333333 0.66666667]]


In [67]:
cluster_priority = df_2.loc[0]['y_kmeans']
cluster_priority

1

### Cosine Similarity & Output Extraction

In [68]:
import numpy as np
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

# Feature selection, removing non-useful rows.
def processing(df_2):
    # Create a feature matrix
    feature_matrix = df_2.drop(['id', 'name', 'artists', 'song_id_name', 'language', 'langauge', 'year', 'supersetted_genres', 'y_kmeans'], axis=1)

    # Apply normalization to the entire feature matrix
    feature_matrix = scaler.fit_transform(feature_matrix)

    # This is the input song. We are isolating it from the dataframe first.
    input_song = feature_matrix[0]
    
    arr = []

    # Returns comparison value of first song in dataset to all others in an array.
    # DataFrame can be customized before Feature Selection to change which song the subject of comparison should be.
    for i in range(1, len(feature_matrix)):
        # Take the i-th song from the feature matrix
        compare_song = feature_matrix[i]
        # Perform cosine similarity.
        cosine_sim = cosine_similarity([input_song], [compare_song])
        cs_value = cosine_sim[0][0]  # Take the single similarity value between the input song and the i-th song.
        arr.append(cs_value)

    return arr

arr = processing(df_2)
arr

[0.8557792849138771,
 0.44333466621440965,
 0.6217919538597342,
 0.6595351337996831,
 0.5325921190316287,
 0.5903300121696766,
 0.585387659940316,
 0.6587891877422413,
 0.47898649698097984,
 0.5902646793184488,
 0.6605904943922329,
 0.6495031471640507,
 0.6008451770397563,
 0.8763219518673233,
 0.6681981773529444,
 0.6990858824901451,
 0.6624208026686018,
 0.6733849120264304,
 0.5491447820774347,
 0.5432289363923403,
 0.56572806945753,
 0.5341361886615004,
 0.564900355029784,
 0.5470116652590753,
 0.6174952842485257,
 0.570854904304482,
 0.6503162917824579,
 0.5951297975308547,
 0.4724655919857847,
 0.6141836115022645,
 0.4609897304040858,
 0.4895476146888606,
 0.6115942725043833,
 0.7040062246539703,
 0.5422497498681239,
 0.431802855055707,
 0.6402951634424082,
 0.8219239953749119,
 0.6390032832679541,
 0.5818958209931389,
 0.404642990683567,
 0.6490182668246877,
 0.5885862915192731,
 0.49894502162135423,
 0.5880709108974047,
 0.4172151501770171,
 0.6330813591237687,
 0.41147366391851

In [69]:
max(arr[1:])

0.9743827002463243

In [70]:
def compile_suggestions(arr):
    suggestions = []
    
    # Compiling all song ids that are above the pre-determined threshold for 'like the first song'.
    for i in range(len(arr)):
        if arr[i] > 0.85:
            suggestions.append(i)
    
    print(suggestions)
    return suggestions
suggestions = compile_suggestions(arr)

[0, 13, 117, 153, 186, 221, 243, 272, 281, 448, 688, 692, 703, 715, 719, 732, 740, 747, 771, 777, 785, 814, 816, 822, 851, 869, 879, 883, 887, 901, 915, 920, 934, 956, 964, 970, 1041, 1228, 1615, 1881, 2143, 2174, 2269, 2425, 2567, 2592, 2706, 2793, 2865, 2881, 3052, 3098, 3174, 3255, 3266, 3652, 3945, 3978, 4090, 4134, 4152, 4164, 4175, 4244, 4290, 4329, 4333, 4353, 4383, 4392, 4565, 4787, 4904, 4907, 4964, 5000, 5167, 5263, 5333, 5481, 5589, 5629, 5805, 5927, 5928, 5971, 6128, 6134, 6247, 6354, 6431, 6432, 6446, 6489, 6557, 6741, 6781, 6801, 6930, 6970, 7021, 7087, 7162, 7178, 7238, 7302, 7373, 7445, 7512, 7587, 7653, 7670, 7713, 7800, 7811, 7902, 7925, 8013, 8015, 8073, 8355, 8386, 8543, 8669, 8811, 8855, 8909, 8943, 8999, 9093, 9099, 9169, 9175, 9269, 9318, 9390, 9529, 9579, 9642, 9671, 9810, 9836, 9957, 9987, 10072, 10363, 10399, 10405, 10437, 10456, 10561, 10572, 10666, 10739, 10765, 11109, 11145, 11162, 11320, 11388, 11391, 11445, 11492, 11495, 11521, 11530, 11551, 11558, 11587,

In [71]:
len(suggestions)

2726

In [72]:
# Extracting out the end result information.
df_main = df_2[df_2.index.isin(suggestions)]

df_main.head()

,id,name,duration_ms,artists,danceability,energy,loudness,mode,speechiness,acousticness,...,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11,standardized_year,y_kmeans
0,5KsMzI24fitxacmN4NABMC,当你老了,198263,[Karen Mok],0.0,0.03435,0.080590,0.1,0.001815,0.1860,...,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.714286,1
13,3j60Gwb3quixJNGZgRWZ3a,The End of the Rainbow - Stereo; 2008 Remaster,134173,0,0.0,0.05610,0.073112,0.1,0.001320,0.1546,...,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.380952,1
117,4ALykgScAe85aRLdIAHQxC,Gee Baby Ain't I Good To You - 2006 Digital Re...,237267,0,0.0,0.04305,0.080179,0.1,0.003025,0.1518,...,0.0,0.0,0.0,0.05,0.0,0.0,0.00,0.0,0.285714,1
186,55wuiF8KoGXTqPo2xpiqk7,How the Other Half Lives,133467,0,0.0,0.04170,0.073166,0.1,0.001485,0.1492,...,0.0,0.0,0.0,0.00,0.0,0.0,0.05,0.0,0.523810,1
221,266HplSXbCxmg5dlqFwtgu,The Thrill of My Life,154293,0,0.0,0.04305,0.074512,0.1,0.002610,0.1634,...,0.0,0.0,0.0,0.05,0.0,0.0,0.00,0.0,0.523810,1


In [73]:
len(df_main) # There is one less for the last 3 features as the input song did not go through the extracted year's steps.
# df_final['standardized_year'].describe()

2477

In [74]:
df_main = df_main.iloc[1:]

df_main = df_main.drop('artists', axis=1)

id_artist_mapping = df.set_index('id')['artists'].to_dict()

# Add 'artist' column to df1 using the mapping
df_main['artists'] = df_main['id'].map(id_artist_mapping)
df_main.head()

,id,name,duration_ms,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,...,key_5,key_6,key_7,key_8,key_9,key_10,key_11,standardized_year,y_kmeans,artists
13,3j60Gwb3quixJNGZgRWZ3a,The End of the Rainbow - Stereo; 2008 Remaster,134173,0.0,0.05610,0.073112,0.1,0.001320,0.1546,0.000042,...,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.380952,1,['Gerry & The Pacemakers']
117,4ALykgScAe85aRLdIAHQxC,Gee Baby Ain't I Good To You - 2006 Digital Re...,237267,0.0,0.04305,0.080179,0.1,0.003025,0.1518,0.000085,...,0.0,0.00,0.05,0.0,0.0,0.00,0.0,0.285714,1,"['The Hoochie Coochie Men', 'Long John Baldry']"
186,55wuiF8KoGXTqPo2xpiqk7,How the Other Half Lives,133467,0.0,0.04170,0.073166,0.1,0.001485,0.1492,0.000000,...,0.0,0.00,0.00,0.0,0.0,0.05,0.0,0.523810,1,['Bill Anderson']
221,266HplSXbCxmg5dlqFwtgu,The Thrill of My Life,154293,0.0,0.04305,0.074512,0.1,0.002610,0.1634,0.000000,...,0.0,0.00,0.05,0.0,0.0,0.00,0.0,0.523810,1,['Bill Anderson']
243,0QDXnXr2VzlxApoxKxrnlE,No Song to Sing,116427,0.0,0.05985,0.075060,0.1,0.002875,0.1394,0.000002,...,0.0,0.05,0.00,0.0,0.0,0.00,0.0,0.523810,1,['Bill Anderson']


### Hard Category Filtering

In [75]:
# Keeping only the songs that match the input song's language.
df_filter = pd.DataFrame()

# Filter rows where the genre is present in the 'supersetted_genres' column
language_filter = df_main[df_main['language'].apply(lambda x: language_prority == x)]
# Concatenate filtered DataFrame with df_filter
df_filter = pd.concat([df_filter, language_filter])
df_filter.shape

(32, 35)

In [76]:
# Keeping only the songs that match the input song's genre(s)
# Iterate over each genre in the priority list
for genre in genre_priority:
    # Filter rows where the genre is present in the 'supersetted_genres' column
    genre_filter = df_filter[df_main['supersetted_genres'].apply(lambda x: genre in x)]
    # Concatenate filtered DataFrame with df_filter
    df_filter = pd.concat([df_filter, genre_filter])
df_filter.shape

C:\Users\yukwa\AppData\Local\Temp\ipykernel_23684\3448013873.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  genre_filter = df_filter[df_main['supersetted_genres'].apply(lambda x: genre in x)]


(32, 35)

In [77]:
prev_years = year_priority - 3
future_years = year_priority + 3

df_filter = df_filter[(df_filter['year'] >= prev_years) & (df_filter['year'] <= future_years)]

In [78]:
df_filter = df_filter[df_filter['y_kmeans'] == cluster_priority]

In [79]:
print(df_filter.shape)
df_filter.drop_duplicates(subset=["id"], keep='first', inplace=True)
print(df_filter.shape)

(10, 35)
(10, 35)


In [80]:
df_filter[['id', 'name',  'artists', 'year', 'language', 'supersetted_genres', 'y_kmeans']]

,id,name,artists,year,language,supersetted_genres,y_kmeans
69477,4QwpHHbsFbBq248fcsnyvZ,廣州足浴一夜,['my little airport'],2012,zh,"[chinese indie, hong kong indie]",1
69712,6j4AAcbeR7Z4zfUKl2EndY,裂縫中的陽光,['JJ Lin'],2013,zh,[],1
69728,1cksZJZGiimTwiKAXyvsBM,千分之一,['Alien Huang'],2013,zh,[],1
122271,3TX2XpgxnfYdCz0trg2ugf,海绵宝宝,['回音哥'],2012,zh,[],1
122734,5UWa3JbaQCcHNmfWa30Lm8,停格,['Tanya Chua'],2015,zh,[chinese indie],1
122737,3yczcQ06p8YXIhymUZQQAg,一個人,['韓紅'],2015,zh,[],1
122781,7yjh1szZFwg7vIzB8Nzhm6,張三的歌,['Jam Hsiao'],2015,zh,[],1
123002,1ePm87d90LN1bEINU2mW1R,偷走,['Ricky Hsiao'],2016,zh,[],1
123022,7MLTCVNXFz5SFN84HSqYVg,日常,['EggPlantEgg'],2017,zh,"[taiwan indie, taiwanese indie]",1
144543,6ZbTj3oFHbNDQDtoO6jSHQ,星月,['Mavis Hee'],2015,zh,[],1


### Recommendations Output

In [81]:
# Generating recc_num of the matched songs at random.
def recommend(df_filter, recc_num):
    recs = []
    if len(df_filter) == 0:
        print("Sorry! There are no songs similar enough to " + input_song + "!")
    elif len(df_filter) < recc_num:
        recc_num = len(df_filter)
        print("Uh oh! We only found " + str(recc_num) + " recommendations!")
    print('Here are your recommendations!')
    for c in range(recc_num):
        recc = df_filter.sample().to_dict()
        name = list(recc['name'].values())[0]
        artists = (list(recc['artists'].values())[0])[1:len(list(recc['artists'].values())[0])-1]
        year = str(list(recc['year'].values())[0])
        id = str(list(recc['id'].values())[0])
        print(str(c + 1) + '. ' + name + ' by ' + artists + ' published in ' + year)
        df_filter = df_filter.drop(list(recc['id'])[0])
        recs.append((id, name, year))
    return recs
# May want to try a filter for repreventing duplicate songs in the list.
recs_for_user = recommend(df_filter, recc_num) # End Product

Here are your recommendations!
1. 海绵宝宝 by '回音哥' published in 2012
2. 停格 by 'Tanya Chua' published in 2015
3. 一個人 by '韓紅' published in 2015
4. 星月 by 'Mavis Hee' published in 2015
5. 偷走 by 'Ricky Hsiao' published in 2016
6. 廣州足浴一夜 by 'my little airport' published in 2012
7. 張三的歌 by 'Jam Hsiao' published in 2015
8. 日常 by 'EggPlantEgg' published in 2017
9. 千分之一 by 'Alien Huang' published in 2013
10. 裂縫中的陽光 by 'JJ Lin' published in 2013


In [83]:
user_id = sp.current_user()["id"]
song_uris = []
for song in recs_for_user:
    result = sp.track(song[0])
    print(result)
    try:
        uri = result["uri"]
        song_uris.append(uri)
    except IndexError:
        print(f"{song[0]} doesn't exist in Spotify. Skipped.")

playlist = sp.user_playlist_create(user=user_id, name=input_playlist, public=False)
sp.playlist_add_items(playlist_id=playlist["id"], items=song_uris)

{'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0t9JGWaRH72nt2UIWSE5CH'}, 'href': 'https://api.spotify.com/v1/artists/0t9JGWaRH72nt2UIWSE5CH', 'id': '0t9JGWaRH72nt2UIWSE5CH', 'name': '回音哥', 'type': 'artist', 'uri': 'spotify:artist:0t9JGWaRH72nt2UIWSE5CH'}], 'available_markets': [], 'external_urls': {'spotify': 'https://open.spotify.com/album/2QYRgRsopNbR3oxaCS6g42'}, 'href': 'https://api.spotify.com/v1/albums/2QYRgRsopNbR3oxaCS6g42', 'id': '2QYRgRsopNbR3oxaCS6g42', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/ab67616d0000b27320144defe95b218bdf797c5a', 'width': 640}, {'height': 300, 'url': 'https://i.scdn.co/image/ab67616d00001e0220144defe95b218bdf797c5a', 'width': 300}, {'height': 64, 'url': 'https://i.scdn.co/image/ab67616d0000485120144defe95b218bdf797c5a', 'width': 64}], 'name': '回音Echo', 'release_date': '2012-07-25', 'release_date_precision': 'day', 'total_tracks': 10, 'type': 'album', 'uri': 'spotify:album:2

HTTP Error for POST to https://api.spotify.com/v1/playlists/6TAdR27IRfr6y5fEOVBGHt/tracks with Params: {'position': None} returned 403 due to Insufficient client scope.


SpotifyException: http status: 403, code:-1 - https://api.spotify.com/v1/playlists/6TAdR27IRfr6y5fEOVBGHt/tracks:
 Insufficient client scope., reason: None